In [1]:
import boto3
import json

# CORREGIDO: Usar guiones estándar (-) en lugar de signos menos (-
bedrock = boto3.client(service_name='bedrock-runtime')
bedrock_agent = boto3.client(service_name='bedrock-agent')

MODEL_ID = "amazon.nova-micro-v1:0"

In [2]:
temperature = .7

inference_config = {"temperature": temperature}

system_prompts = [{"text": "You are a virtual travel assistant that suggests destinations based on user preferences."
                + "Only return destination names and a brief description."}]

messages = []

message_1 = {
    "role": "user",
    "content": [{"text": "Create a list of 3 travel destinations."}]
}

messages.append(message_1)

response = bedrock.converse(
    modelId=MODEL_ID,
    messages=messages,
    system=system_prompts,
    inferenceConfig=inference_config
)

def print_response(response):
    model_response = response.get('output', {}).get('message', {}).get('content', [{}])[0].get('text', '')

    print("✈️ Your suggested travel destinations:")
    print(model_response)

print_response(response)

✈️ Your suggested travel destinations:
1. **Kyoto, Japan**  
   - Experience traditional Japanese culture with stunning temples, serene gardens, and historic tea houses.

2. **Santorini, Greece**  
   - Enjoy breathtaking views of the Aegean Sea, white-washed buildings, and stunning sunsets over volcanic beaches.

3. **Banff, Canada**  
   - Explore the majestic Rocky Mountains, crystal-clear lakes, and abundant wildlife in this picturesque national park.


In [3]:
message_2 = {
        "role": "user",
        "content": [{"text": "Only suggest travel locations that are no more than one short flight away."}]
}

messages.append(message_2)

response = bedrock.converse(
    modelId=MODEL_ID,
    messages=messages,
    system=system_prompts,
    inferenceConfig=inference_config
)

print_response(response)

✈️ Your suggested travel destinations:
1. **Orlando, Florida**  
   Experience world-famous theme parks like Disney World, Universal Studios, and SeaWorld.

2. **Miami, Florida**  
   Enjoy vibrant beaches, rich culture, and lively nightlife.

3. **San Diego, California**  
   Discover beautiful beaches, the San Diego Zoo, and stunning coastal views.


In [4]:
message_3 = {
        "role": "user",
        "content": [{"text": "Only suggest travel locations that are budget-friendly."}]
}

messages.append(message_3)

response = bedrock.converse(
    modelId=MODEL_ID,
    messages=messages,
    system=system_prompts,
    inferenceConfig=inference_config
)

print_response(response)

✈️ Your suggested travel destinations:
1. **Mexico City, Mexico**
   - Vibrant culture, rich history, and affordable street food.

2. **Guatemala City, Guatemala**
   - Mix of ancient ruins, colorful markets, and budget accommodations.

3. **San José, Costa Rica**
   - Eco-friendly adventures, lush national parks, and low-cost travel options.


In [ ]:
# # Eliminar el existente
# bedrock_agent.delete_prompt(promptIdentifier="SQCGTGH0VE")

# # Luego crear el nuevo con el mismo nombre


{'ResponseMetadata': {'RequestId': '4b837d7e-a9cf-4950-a616-804f5cf3250c',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'date': 'Fri, 14 Nov 2025 00:43:32 GMT',
   'content-type': 'application/json',
   'content-length': '19',
   'connection': 'keep-alive',
   'x-amzn-requestid': '4b837d7e-a9cf-4950-a616-804f5cf3250c',
   'x-amz-apigw-id': 'UAgwNHb0oAMEppw=',
   'x-amzn-trace-id': 'Root=1-69167b34-48728723594277ca082aa2df'},
  'RetryAttempts': 0},
 'id': 'SQCGTGH0VE'}

In [8]:
try:
    response = bedrock_agent.create_prompt(
        name="Travel-Agent-Prompt",
        description="Checks if all trip information has been provided.",
        variants=[
            { 
                "name": "Variant1",
                "modelId": MODEL_ID,
                "templateType": "CHAT",
                "inferenceConfiguration": {
                    "text": {
                        "temperature": 0.4
                    }
                },
                "templateConfiguration": { 
                    "chat": {
                        'system': [ 
                            {
                                "text": """You are a travel agent evaluating trip requests for custom itineraries. 
                                Review the message carefully and answer YES or NO to the following screening questions. 
                                Be strict—if any detail is missing or unclear, answer NO.

                                A) Is the destination clearly stated?
                                B) Are the travel dates within a reasonable range (not last−minute or over a year away)?
                                C) Does the request avoid high−risk or restricted activities (e.g., extreme sports, off−grid travel)?
                                D) Is there any mention of a valid passport or travel documentation?
                                E) Is there enough information to follow up with a proposed itinerary?"""
                            }
                        ],
                        'messages': [{
                            'role': 'user',
                            'content': [ 
                                {
                                    'text': "Trip request: {{event_request}}"
                                }
                            ]
                        }],
                        'inputVariables' : [
                            { 'name' : 'event_request'}
                        ]
                    }
                }
        }]
    )
    print("Created!")
    prompt_arn = response.get("arn")
except bedrock.exceptions.ConflictException as e:
    print("Already exists!")
    response = bedrock.list_prompts()
    prompt = next((prompt for prompt in response['promptSummaries'] if prompt['name'] == "TripBooker_xyz"), None)
    prompt_arn = prompt['arn']

prompt_arn

Created!


'arn:aws:bedrock:us-east-1:613468529489:prompt/6AFUSV35LA'

In [9]:
response = bedrock.converse(
    modelId=prompt_arn,
    promptVariables={
        'event_request': {
            'text': """
                Hi there! I'm planning a trip to Italy with my partner and would love some help organizing the itinerary. We're hoping to travel between September 10-20 this year, ideally flying into Rome and spending a few days in Florence and Venice as well. We’d love recommendations on tours, cultural sites, and good local restaurants. We’re not interested in anything risky like skydiving or hiking remote trails — just want a relaxing and enriching experience. We both have valid passports. Let me know what other details you need!
                """
        }
    },
)
print(response['output']['message']['content'][0]['text'])

YES

A) Is the destination clearly stated?
YES. The destination is clearly stated as Italy.

B) Are the travel dates within a reasonable range (not last−minute or over a year away)?
YES. The travel dates are within a reasonable range, September 10-20 this year.

C) Does the request avoid high−risk or restricted activities (e.g., extreme sports, off−grid travel)?
YES. The request specifies a desire for a relaxing and enriching experience, avoiding risky activities like skydiving or hiking remote trails.

D) Is there any mention of a valid passport or travel documentation?
YES. Both travelers have valid passports.

E) Is there enough information to follow up with a proposed itinerary?
YES. There is enough information provided to start planning an itinerary, including the destination, travel dates, preferred cities (Rome, Florence, Venice), and general interests (tours, cultural sites, local restaurants).
